In [ ]:
from pathlib import Path
import csv
from collections import defaultdict
import os.path
from statistics import mean 
import pandas as pd
import altair as alt
import fiona
import geopandas as gpd
import gpdvega
import json
import datetime
from matplotlib import cm
from matplotlib.colors import to_hex
import shapely
from shapely.geometry import Polygon
import re

colormap = cm.spring

In [ ]:
today = datetime.date.today()
today

In [ ]:
str(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))

In [ ]:
def read_csv(filename):
    with open(filename, newline='') as File:  
        reader = csv.reader(File)
        header1 = next(reader)
        header2 = next(reader)
        results = []
        for row in reader:
            folder = Path(row[0].strip("'"))
            parts = folder.parts
            if len(parts) > 2 and parts[-1].lower().endswith('.tif'):
                results.append({"folder": str(folder.parent), "hits": int(row[1]), "bytes": int(row[2]), "cost": float(row[3])})
        return results

In [ ]:
def read_json(filename):
    with open(filename, encoding='utf-8') as File:  
        reader = json.loads(File.read())
        reader_files = reader['Files']
        today = datetime.date.today()  
        results = []
        for i,v in reader_files.items():
            if str(i).endswith('.TIF') or str(i).endswith('.tif') or str(i).endswith('.tiff'):
                results.append({"folder": str(i), "hits": int(v[0]), "bytes": int(v[1]), 'date':today})
        return results

In [ ]:
file_namee="201909.json"

In [ ]:
file_date = file_namee.split(".")[0][:4]+ "-" + file_namee.split(".")[0][4:]

In [ ]:
file_date+ '-01'

In [ ]:
all_entries = read_json(file_namee)

In [ ]:
len(all_entries)

In [ ]:
def tile_index_from_path(path):
    def go(part):
        pattern = r'(?P<key>[xy])_(?P<num>.*)'
        match = re.match(pattern, part, re.IGNORECASE)
        if match is None:
            return {}
        gd = match.groupdict()
        return {gd['key']: gd['num']}
    
    result = {}
    for part in path.split('/'):
        result.update(go(part))
    return ((result['x']), (result['y']))


def product_name(folder):
    parts = Path(folder).parts
    if parts[0] == 'mangrove_cover':
        return parts[0]
    return os.path.join(*parts[:2])


def spatial_id(folder):
    parts = Path(folder).parts
    print(parts)
    print(parts[-1])
    if parts[-2] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        print("OK")
    if len(parts) > 2 and parts[0] == 'L2' and parts[1] == 'sentinel-2-nrt' and parts[-2] in ['NBAR', 'NBART', 'QA', 'SUPPLEMENTARY', 'LAMBERTIAN']:
        try:
            print(parts[-3].split("_")[-2][1:])
            return parts[-3].split("_")[-2][1:]
        except IndexError:
            print(folder)
    if len(parts) >2 and parts[0] == 'hltc' or parts[0] == 'item_v2':
        try:
            print(parts[-1].split("_")[2])
            return parts[-1].split("_")[2]
        except IndexError:
            print(folder)
    if len(parts) >2 and parts[0] == 'nidem':
        try:
            print(parts[-1].split("_")[1])
            return parts[-1].split("_")[1]
        except IndexError:
            print(folder)
    elif len(parts) >2 and parts[0]=='bare-earth' or parts[0]=='geomedian-australia' or parts[0]=='WOfS' or parts[0]=='fractional-cover':
        return ','.join(tile_index_from_path(folder))
    elif len(parts) >2 and parts[0] == 'projects' or parts[0] == 'weathering-intensity':
        return ' '
    elif len(parts) >2 and parts[0] == 'multi-scale-topographic-position':
        return ' '
    else:
        return '<none>'


def coordsgeojson(spatialid):
    with open('australian-mgrs-tiles.geojson') as fl:
        input_gj = json.load(fl)
    feats = input_gj['features']
    for feat in feats:
        if 'MGRS' in feat['properties']:
            mgrs = feat['properties']['MGRS']
            if spatial_id == mgrs:
                polygon = Polygon(feat["geometry"]['coordinates'][0])
                lat = round(polygon.centroid.y, 1)
                lon = round(polygon.centroid.x, 1) 
                return lat
    
    
    
def latcord(folder, spatialid):
    parts = Path(folder).parts

    if len(parts) >2 and parts[0] == 'hltc' or parts[0] == 'item_v2':

        return (parts[-1].split('.tif'))[0].split("_")[4]
    elif len(parts) >2 and parts[0] == 'nidem':
        return parts[-1].split("_")[3]
    elif len(parts) >2 and parts[0] == 'projects' or parts[0] == 'weathering-intensity':
        return ' '
    elif len(parts) >2 and parts[0] == 'multi-scale-topographic-position':
        return ' '
    else:
        return '<none>'

    
def loncord(folder, spatialid):
    parts = Path(folder).parts
    if len(parts) >2 and parts[0] == 'hltc' or parts[0] == 'item_v2':
        print(parts[-1].split("_")[3])
        return parts[-1].split("_")[3]
    elif len(parts) >2 and parts[0] == 'nidem':
        return parts[-1].split("_")[2]
    elif len(parts) >2 and parts[0] == 'projects' or parts[0] == 'weathering-intensity':
        return ' '
    elif len(parts) >2 and parts[0] == 'multi-scale-topographic-position':
        return ' '
    else:
        return '<none>'


def merge_pre(folder, dicts):
    today = datetime.date.today() 
    return {
        'hits': max(int(d['hits']) for d in dicts),
        'bytes/GB': round(sum(int(d['bytes']) for d in dicts)/1000000000,2),
        #'cost': sum(float(d['cost']) for d in dicts),
        'spatial_id': spatial_id(folder),
        'product': product_name(folder),
        #'folder': folder,
        #date': str(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))
        'date': file_date + '-01',
        'Lat' : latcord(folder, spatial_id(folder)),
        'Lon' : loncord(folder, spatial_id(folder))
    }

def group(entry_list, key):
    lookup = defaultdict(list)
    
    for d in entry_list:
        lookup[d[key]].append(d)
        
    return lookup
    

def merge(dicts):
    return {
        'hits': sum(int(d['hits']) for d in dicts),
        'bytes': sum(int(d['bytes']) for d in dicts),
        #'cost': sum(float(d['cost']) for d in dicts),
    }

In [ ]:
stage2 = [merge_pre(key, value) for key, value in group(all_entries, 'folder').items()]

In [ ]:
stage2


In [ ]:
products = [d for d in stage2]

In [ ]:
products

In [ ]:
with open('australian-mgrs-tiles.geojson') as fl:
    input_gj = json.load(fl)

In [ ]:
with open('albers_grid.geojson') as fl:
    input_gj_2 = json.load(fl)

In [ ]:
feats2= input_gj_2['features']

In [ ]:
feats = input_gj['features']

In [ ]:
for feat in feats2:
    if 'label' in feat['properties']:
        label = feat['properties']['label']
        for dict_item in products:
            if (dict_item['spatial_id']) == label:
                polygon = Polygon(feat["geometry"]['coordinates'][0]) 
                dict_item['Lat'] = round(polygon.centroid.y, 1)
                dict_item['Lon'] = round(polygon.centroid.x, 1) 

In [ ]:
for feat in feats:
    if 'MGRS' in feat['properties']:
        mgrs = feat['properties']['MGRS']
        count=0
        for dict_item in products:
            if (dict_item['spatial_id']) == mgrs:
                polygon = Polygon(feat["geometry"]['coordinates'][0])
                dict_item['Lat'] = round(polygon.centroid.y, 1)
                dict_item['Lon'] = round(polygon.centroid.x, 1) 

In [ ]:
products

In [ ]:
sentinel2 = [d for d in stage2 if d['product'] == 'L2/sentinel-2-nrt']

## 

In [ ]:
sentinel2[0]

In [ ]:
keys = sentinel2[0].keys()

In [ ]:
keys

In [ ]:
with open('newdata.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(products)

In [ ]:
with open('newdata.csv', 'a') as f:
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writerows(products)
